In [1]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Create Database from Parquet") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Create a database
spark.sql("CREATE DATABASE IF NOT EXISTS goodreads_reviews")

# Select the database
spark.sql("USE goodreads_reviews")

# Save DataFrame as a table in the database
spark.sql("CREATE TABLE IF NOT EXISTS reviews USING parquet OPTIONS (PATH 'C:/Users/jose/Downloads/goodreads_reviews_parquet')")

DataFrame[]

## Table Check:

In [2]:
# Show tables in the current schema
spark.sql("SHOW TABLES").show()

+-----------------+---------+-----------+
|        namespace|tableName|isTemporary|
+-----------------+---------+-----------+
|goodreads_reviews|  reviews|      false|
+-----------------+---------+-----------+



In [3]:
# Use the database
spark.sql("USE goodreads_reviews")

# Query the data from the table
result = spark.sql("SELECT * FROM reviews")

# Show the result
result.show()

+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
| book_id|          date_added|        date_updated|n_comments|n_votes|rating|             read_at|           review_id|         review_text|          started_at|             user_id|
+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|25913122|Tue Dec 29 13:45:...|Fri Aug 26 11:01:...|         2|      5|     4|Tue Jul 12 00:00:...|6083f89f3e7ea38c4...|Compre este libro...|Tue Jul 12 00:00:...|2ce9cf233c1503ed5...|
|27823971|Wed Oct 28 07:51:...|Wed Feb 17 21:02:...|         1|      2|     3|Wed Feb 17 00:00:...|28b70411d46da4471...|No puedo ser impa...|Sat Feb 13 00:00:...|2ce9cf233c1503ed5...|
|25796670|Sat Oct 03 14:14:...|Wed Mar 16 00:51:...|         1|      4|     2|Tu

## Passing *Spark*  `result` into a pandas dataframe for modelling, first we will drop some columns to avoid the error:
## ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it.

In [4]:
# Assuming df is your Spark DataFrame
columns_to_drop = ["date_added", "date_updated","n_comments","n_votes","review_id","started_at","read_at"]  # List of columns to drop

# Drop the specified columns
result = result.drop(*columns_to_drop)

# Show the result
result.show()

+--------+------+--------------------+--------------------+
| book_id|rating|         review_text|             user_id|
+--------+------+--------------------+--------------------+
|25913122|     4|Compre este libro...|2ce9cf233c1503ed5...|
|27823971|     3|No puedo ser impa...|2ce9cf233c1503ed5...|
|25796670|     2|Habia desistido d...|2ce9cf233c1503ed5...|
|25778324|     5|Llegue a este lib...|2ce9cf233c1503ed5...|
|18712886|     2|Confieso desde un...|2ce9cf233c1503ed5...|
|28963557|     5|           Perfecto.|2ce9cf233c1503ed5...|
|25469092|     4|Aunque me gusto m...|2ce9cf233c1503ed5...|
|22037944|     4|Que dificil es re...|2ce9cf233c1503ed5...|
|18499402|     4|Quise leer este l...|2ce9cf233c1503ed5...|
|26798985|     4|En la resena ante...|2ce9cf233c1503ed5...|
|25432891|     4|Bueno, voy a ser ...|2ce9cf233c1503ed5...|
| 3373897|     2|Estuvo bien, pero...|2ce9cf233c1503ed5...|
|16080337|     4|Cuando vi este li...|2ce9cf233c1503ed5...|
|18076991|     4|I liked this one ...|2c

In [5]:
from pyspark.sql import *  
from pyspark.sql.functions import *  
from pyspark.sql.types import *  
import numpy as np    
import pandas as pd


reviews_pandas_df= result.select("*").toPandas()

Py4JJavaError: An error occurred while calling o56.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 3.0 failed 1 times, most recent failure: Lost task 3.0 in stage 3.0 (TID 6) (DESKTOP-IK25UHA executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFa

In [6]:
# Assuming df is your Spark DataFrame
columns_to_drop = ["review_text"]  # List of columns to drop

# Drop the specified columns
result = result.drop(*columns_to_drop)

# Show the result
result.show()

+--------+------+--------------------+
| book_id|rating|             user_id|
+--------+------+--------------------+
|25913122|     4|2ce9cf233c1503ed5...|
|27823971|     3|2ce9cf233c1503ed5...|
|25796670|     2|2ce9cf233c1503ed5...|
|25778324|     5|2ce9cf233c1503ed5...|
|18712886|     2|2ce9cf233c1503ed5...|
|28963557|     5|2ce9cf233c1503ed5...|
|25469092|     4|2ce9cf233c1503ed5...|
|22037944|     4|2ce9cf233c1503ed5...|
|18499402|     4|2ce9cf233c1503ed5...|
|26798985|     4|2ce9cf233c1503ed5...|
|25432891|     4|2ce9cf233c1503ed5...|
| 3373897|     2|2ce9cf233c1503ed5...|
|16080337|     4|2ce9cf233c1503ed5...|
|18076991|     4|2ce9cf233c1503ed5...|
|18483104|     3|2ce9cf233c1503ed5...|
|20560907|     3|2ce9cf233c1503ed5...|
|17661625|     4|2ce9cf233c1503ed5...|
|20613470|     5|2ce9cf233c1503ed5...|
|22037901|     5|2ce9cf233c1503ed5...|
|13266803|     5|2ce9cf233c1503ed5...|
+--------+------+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql import *  
from pyspark.sql.functions import *  
from pyspark.sql.types import *  
import numpy as np    
import pandas as pd


reviews_pandas_df= result.select("*").toPandas()

In [ ]:
reviews_pandas_df

In [ ]:
# Stop SparkSession
spark.stop()